## Распознавание текста из файла с указанием ссылки на него

Устанавливаем необходимые библиотеки

In [44]:
# pip install -U openai-whisper

In [43]:
import whisper

model = whisper.load_model("medium")

# загрузите аудиофайл, здесь необходимо указать путь к вашему файлу!!!
audio = whisper.load_audio("/content/drive/MyDrive/звук/1.wav")

options = {
    "language": "RU", # поставьте необходимый язык
}
result = whisper.transcribe(model, audio, **options)
# print(result["text"])

# Записываем файл
with open("output.txt", "w", encoding="utf-8") as file:
    file.write(result["text"])

print("Результат записан в файл 'output.txt'")

Результат записан в файл 'output.txt'


## Распознавание текста из файла с интерактивной загрузкой

In [41]:
# pip install fileupload

In [68]:
import ipywidgets as widgets
from IPython.display import display, HTML
import time  # Добавлено для имитации процесса

# Создаем виджет загрузки файла, который принимает как WAV, так и MP3/M4A файлы
uploader = widgets.FileUpload(accept='.wav,.mp3,.m4a', multiple=False)

# Создаем виджет прогресс-бара
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Распознавание:')
output = widgets.Output()

# Создаем функцию для обработки загрузки файла и выполнения транскрибации
def recognize_speech(change):
    uploaded_filename = next(iter(uploader.value))
    audio_data = uploader.value[uploaded_filename]['content']
    file_extension = uploaded_filename.split('.')[-1].lower()  # Получаем расширение файла

    # Сохраняем загруженный файл на диск с правильным расширением
    with open(f"uploaded_audio.{file_extension}", "wb") as audio_file:
        audio_file.write(audio_data)

    # Отображаем прогресс-бар
    with output:
        display(progress_bar)

    # Имитируем процесс транскрибации (замените этот участок на фактический код транскрибации)
    for progress in range(0, 101, 10):
        time.sleep(1)  # Имитация времени выполнения
        progress_bar.value = progress / 100.0

    # Загружаем модель и аудио для транскрибации
    model = whisper.load_model("medium")
    audio = whisper.load_audio(f"uploaded_audio.{file_extension}")
    options = {
        "language": "RU", # Язык ввода, если не указан, то определяется автоматически
    }
    result = whisper.transcribe(model, audio, **options)

    # Отображаем результат транскрибации
    with output:
        progress_bar.value = 1.0  # Устанавливаем прогресс-бар на 100% после завершения
        # print(f"<b>Результат транскрибации:</b> {result['text']}")

        # Записываем результат в файл
        with open('result_full.txt', 'w', encoding='utf-8') as file:
            file.write(result["text"])
        print("Результат был сохранен в файл 'result_full.txt'.")

# Устанавливаем обработчик событий для загрузки файла
uploader.observe(recognize_speech, names='value')

# Отображаем виджет загрузки файла и вывод
display(uploader)
display(output)




FileUpload(value={}, accept='.wav,.mp3,.m4a', description='Upload')

Output()

## Распознавание голоса с микрофона из google colab


In [34]:
# pip install SpeechRecognition

In [33]:
# !apt-get install -y python3-pyaudio

In [35]:
# pip install pyaudio

In [36]:
# pip install pipwin

In [37]:
# !pip install ffmpeg-python

In [26]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

# HTML-код для создания кнопки записи аудио
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Нажмите для начала записи");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    mimeType : 'audio/webm;codecs=opus'
  };
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
};

recordButton.innerText = "Идет запись... нажмите, чтобы остановить";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Сохранение записи... пожалуйста, подождите!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  resolve(base64data.toString())
});
}
});
</script>
"""

def get_audio():
  # Отображаем HTML-код для записи аудио
  display(HTML(AUDIO_HTML))

  # Получаем данные аудио из JavaScript
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  # Преобразуем данные аудио в WAV-формат
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  # Разбираем заголовок WAV-файла для получения частоты дискретизации и аудиоданных
  riff_chunk_size = len(output) - 8
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  riff = output[:4] + bytes(b) + output[8:]

  srate, audio = wav_read(io.BytesIO(riff))

  return audio, srate


In [28]:
# вызов функции get_audio
audio, srate = get_audio()

In [30]:
import numpy as np
from scipy.io.wavfile import write, read
import speech_recognition as sr

# Сохраните данные аудио в файл
write('temp_audio.wav', srate, audio)

# функция для распознавания аудио
def transcribe_audio(audio_file):
    r = sr.Recognizer()
    try:
        with sr.AudioFile(audio_file) as source:
            # Попробуйте распознать аудио
            audio_data = r.record(source)
            query = r.recognize_google(audio_data, language='ru-RU')
            return query.lower()
    except sr.UnknownValueError:
        # Если не удалось распознать аудио
        return "Не удалось распознать аудио"
    except sr.RequestError as e:
        # Если возникла ошибка при запросе к сервису распознавания
        return f"Ошибка при запросе к сервису распознавания: {str(e)}"

# Вызов функции для распознавания
result = transcribe_audio('temp_audio.wav')

# Вывод результата распознавания
print(f'Вы сказали: {result}')



Вы сказали: привет я хотела сказать что сегодня очень классная погода


In [32]:
# Запись результата в файл result.txt
with open('result.txt', 'w', encoding='utf-8') as file:
    file.write(result)

## Распознавание голоса с микрофона не из google colab

In [ ]:
import speech_recognition as sr

# Определяем функцию для записи аудио с микрофона и его распознавания
def record_volume():
    r = sr.Recognizer()  # Создаем объект Recognizer для управления распознаванием речи

    # Начинаем запись аудио с микрофона
    with sr.Microphone(device_index=1) as source:  # Указываем индекс устройства (может понадобиться указать свой)
        print('Слушаю...')  # Выводим сообщение о начале записи

        audio = r.listen(source)  # Записываем аудио и сохраняем в переменной audio

    query = r.recognize_google(audio, language='ru-RU')  # Распознаем аудио с использованием Google Web Speech API
    print(f'Вы сказали: {query.lower()}')  # Выводим распознанный текст в нижнем регистре с сообщением "Вы сказали:"

# Вызываем функцию для записи и распознавания аудио
record_volume()
